# 纵向监测数据在地图上的展示——热力图

## 建立隧道轴线与地图的联系

这个热力图主要实现隧道轴向的监测数据在地图上的展示。

在地图上展示热力图，需要两种元素，其一是隧道纵向上光纤测点的地理坐标（或者经纬度），其二是光纤测点的监测数据。
关键在于，如何将隧道纵向光纤的每个测点，通过坐标转换，一一映射在地图上。

工程方提供的区间平面图CAD中，展示了隧道及周边建筑的平面图。图纸中显示其坐标系统为大连城建坐标系，而我国常用的为北京1954坐标系等。查阅后发现，没有官方明确的坐标转换参数将其转换为常见的坐标系。而尝试使用网上的资料（大连金州以南地区大连城建坐标系采用1954年北京坐标系，高斯投影3度带，中央子午线经度值121°30′，横坐标的加常数30公里）进行参数转换后，还是无法得到常见坐标系下较为准确的坐标。

因此我采用的方法是：在CAD平面图中采集几个标志性建筑物（如中山广场等）的XY坐标，和CAD平面图的隧道轮廓一同导入arcgis中，此时隧道的位置是偏移的，需要修正。与此同时，在arcgis中加载常见坐标系下的世界地图，并在地图中捕捉与CAD中相同的标志性建筑物的位置与XY坐标。最后，使用手动空间修正功能，建立起CAD标志性建筑物（需校正）与地图上标志性建筑物的映射，从而将同时加载的CAD平面图中的隧道轮廓映射到正确的位置。经过角度的微调后，便可以得到隧道轮廓（与轴线）在常见坐标系下的真实位置。

校正位置后的隧道长度发生了变化，大致变为CAD平面图中的1.25倍。因此原CAD平面图中的长度（光纤上的距离、里程等），需要乘1.25的倍数，映射到校正后的隧道上，即完成了CAD工程图代表的真实位置到地图坐标的映射。而隧道的角度并未发生明显变化，可以忽略校正带来的影响。

为简化隧道模型，用多条直线对隧道轮廓中的弧线进行了近似，同时得到了隧道纵向上的关键拐点，算上首尾共计7个点、6段相连的折线。每段折线都有其固定长度，及与水平方向的夹角；每个关键拐点的坐标都是已知的。

纵向的光纤不连续，但每段光纤起始位置在隧道中的里程，和每条光纤上去掉跳线后的有效测量距离，都是已知的。因此，在已知光纤起始里程的情况下，通过乘1.25的倍数，可以转化为校正位置后隧道轴向上的里程；而光纤上的真实距离（即监测数据文件中的Distance项），通过乘1.25的倍数，可以得到校正位置后光纤的距离；通过已知隧道关键拐点的坐标、每个拐点间直线的距离以及和水平方向的夹角，便可以求出纵向光纤的每个测点在地图上的映射坐标。

举例：假设某段纵向光纤在真实隧道上的起始里程是500m。通过和关键拐点比较，可以得知其起始点在第二和第三个拐点之间，此时隧道轴向与水平方向的夹角为126度，第二个拐点在地图上的坐标为(13539641.6028,4710901.8387)，在校正后的隧道的里程（第一段折线长）为415.7159m。忽略跳线的长度，假设这段光纤不经过拐点，光纤上的距离项为d，则可求得光纤测点在地图上（即校正后）的x坐标为：

x=13539641.6028+((500+d)×1.25-415.7159)×cosθ

y坐标同理。

## 将监测数据展示在地图上

地图展示离不开GIS，而为了更方便地进行二次开发，可以选用开源GIS编写程序。这里选用了Python中的folium库，它可以通过特定的代码设计地图，并将地图转换为本地的HTML格式，在PyQt5界面中也可以调用。

下面是相关程序：

In [1]:
# 导入需要调用的库
import pymysql
import numpy as np
import pandas as pd
import math
import folium
from folium.plugins import HeatMap

In [2]:
# 建立与数据库的连接，读取库中存储的数据
conn = pymysql.connect(host='localhost', port=3306, user='root', password='19971209', db='try20201110')
cur = conn.cursor()
sql = 'SELECT distance, strain from 3_1_longstrain_20201201_400_5_10'
cur.execute(sql)
data = cur.fetchall()
cur.close()
conn.close()
array = np.array(data)
array = array.astype(float)

print(array)

[[ 0.00000e+00 -8.36300e+00]
 [ 5.10000e-02 -4.14500e+00]
 [ 1.03000e-01 -1.22540e+01]
 ...
 [ 1.99999e+02  8.87693e+02]
 [ 2.00050e+02 -9.54270e+01]
 [ 2.00100e+02  1.00234e+02]]


In [3]:
# 将监测数据numpy数组转化为pandas的dataframe形式，便于对每列数据进行处理
dataframe = pd.DataFrame(array)
dataframe.columns = ['Distance(m)', 'Data']
dataframe = dataframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
# 原数据中只有“距离”和“监测值”两列，可增设两列用以存储地理信息XY
dataframe.insert(loc=len(dataframe.columns), column='X', value=pd.NA)
dataframe.insert(loc=len(dataframe.columns), column='Y', value=pd.NA)
print(dataframe)

      Distance(m)      Data    X    Y
0           0.000    -8.363  NaN  NaN
1           0.051    -4.145  NaN  NaN
2           0.103   -12.254  NaN  NaN
3           0.154    88.282  NaN  NaN
4           0.205   325.217  NaN  NaN
...           ...       ...  ...  ...
3894      199.896  1249.301  NaN  NaN
3895      199.947     0.000  NaN  NaN
3896      199.999   887.693  NaN  NaN
3897      200.050   -95.427  NaN  NaN
3898      200.100   100.234  NaN  NaN

[3899 rows x 4 columns]


In [4]:
start_mileage = 100  # 光纤起始里程
point_x = [13539822.9102, 13539641.6028, 13539450.0096, 13539272.7948, 13538841.6216, 13538797.8751, 13538645.2470]  # 关键拐点的X坐标
point_y = [4710527.7433, 4710901.8387, 4711160.8930, 4711685.9827, 4714470.3603, 4714601.7306, 4714904.7998]  # 关键拐点的Y坐标
mileages = [0, 415.7159, 737.9225, 1292.1105, 4109.6748, 4248.1374, 4587.4691]  # 每两个关键拐点间的距离，即每段折线的长度
angles = [116, 126, 109, 99, 108, 117]  # 每段折线与水平方向的夹角

In [5]:
# 定义函数，将光纤上的真实距离转换为对应的地理坐标XY
def xSelect(x, xx, d, theta):
    return (xx + (start_mileage * 1.25 + x * 1.25 - d)*math.cos(theta))

def ySelect(y, yy, d, theta):
    return (yy + (start_mileage * 1.25 + y * 1.25 - d)*math.sin(theta))

In [6]:
# 将计算出的地理坐标XY存入监测数据的dataframe变量
# 这里的三角函数必须要先把角度转化为弧度，即math.radians
dataframe['X'] = dataframe['Distance(m)'].apply(xSelect, args=(point_x[0], mileages[0], math.radians(angles[0])))
dataframe['Y'] = dataframe['Distance(m)'].apply(ySelect, args=(point_y[0], mileages[0], math.radians(angles[0])))

In [7]:
# 定义函数，将XY坐标转化为经纬度
# folium库需要获取经纬度而不是地理坐标，因此需要进一步转化
def webMercator2Lng(x):
    lng = x / 20037508.34 * 180
    return lng

def webMercator2Lat(y):
    lat = y / 20037508.34 * 180
    lat = 180 / math.pi * (2 * math.atan(math.exp(lat * math.pi / 180)) - math.pi / 2)
    return lat

In [8]:
# 在监测数据的dataframe变量中新建两列，用于存储计算出的经纬度
dataframe.insert(loc=len(dataframe.columns), column='lat', value=pd.NA)
dataframe.insert(loc=len(dataframe.columns), column='lng', value=pd.NA)
dataframe['lng'] = dataframe['X'].map(webMercator2Lng)
dataframe['lat'] = dataframe['Y'].map(webMercator2Lat)
print(dataframe)

      Distance(m)      Data             X             Y        lat         lng
0           0.000    -8.363  1.353977e+07  4.710640e+06  38.922945  121.629806
1           0.051    -4.145  1.353977e+07  4.710640e+06  38.922945  121.629806
2           0.103   -12.254  1.353977e+07  4.710640e+06  38.922946  121.629806
3           0.154    88.282  1.353977e+07  4.710640e+06  38.922946  121.629806
4           0.205   325.217  1.353977e+07  4.710640e+06  38.922947  121.629805
...           ...       ...           ...           ...        ...         ...
3894      199.896  1249.301  1.353966e+07  4.710865e+06  38.924514  121.628822
3895      199.947     0.000  1.353966e+07  4.710865e+06  38.924515  121.628822
3896      199.999   887.693  1.353966e+07  4.710865e+06  38.924515  121.628822
3897      200.050   -95.427  1.353966e+07  4.710865e+06  38.924516  121.628822
3898      200.100   100.234  1.353966e+07  4.710865e+06  38.924516  121.628821

[3899 rows x 6 columns]


In [9]:
# 加载地图的底图，便于展示所以地图底图选了黑白色，也可以换成其他类型的彩色地图
# 鼠标左键拖动地图，鼠标滚轮缩放地图
m = folium.Map([38.92382434279403, 121.62905943424302], tiles="stamentoner", zoom_start=17)
m

In [10]:
# 加载隧道的轮廓线、隧道两侧的车站轮廓线
# 为减少加载时间，对轮廓线进行了简化。这里显示的是隧道南侧的车站
location1 = [[38.95173013811784, 121.62018973862097], [38.95263161736866, 121.6196048455394], [38.952553501077574, 121.6194003889807], [38.95289265872652, 121.61918048139911], [38.95294793088148, 121.61932519999141], [38.9526629630606, 121.61950360540686], [38.95279517089111, 121.61984963645436], [38.95280389624994, 121.61987245366257]]
location2 = [[38.95280389624994, 121.61987245366257], [38.951849416660195, 121.62049175221954], [38.95172988662253, 121.62018991828401]]
location3 = [[38.92212701134782, 121.63020900831226], [38.924741475011885, 121.62858027287038], [38.926551887204326, 121.62685919061731], [38.93022136392138, 121.6252672861021], [38.94967620893258, 121.6213939302595], [38.95059397793382, 121.62100100715416], [38.952711214581136, 121.61962990853581]]
location4 = [[38.92219240643727, 121.630388312043], [38.924806867691515, 121.62875957660113], [38.926617285203626, 121.62703849434806], [38.93028675154969, 121.6254465000013], [38.94974157862265, 121.62157323399023], [38.95065934677746, 121.62118022105336], [38.952779103374, 121.61980759529906]]
location5 = [[38.9222135967536, 121.630446792368], [38.922247716504316, 121.63054030698909], [38.922534281227, 121.63087519892706], [38.92314711755933, 121.63011504453354], [38.92312446686266, 121.63005288111587], [38.92366894120335, 121.62971745018874], [38.92365111290952, 121.62966858183725], [38.923870433459385, 121.62953338538699], [38.923797023839654, 121.6293321627633], [38.923897437892954, 121.62925984838293], [38.92382434279403, 121.62905943424302]]
location6 = [[38.92209215770444, 121.63011387672366], [38.9222135967536, 121.630446792368], [38.92383938254135, 121.62944822509803]]
location7 = [[38.92382434279403, 121.62905943424302], [38.92209215770444, 121.63011387672366]]

line1 = folium.PolyLine(location1).add_to(m)
line2 = folium.PolyLine(location2).add_to(m)
line3 = folium.PolyLine(location3).add_to(m)
line4 = folium.PolyLine(location4).add_to(m)
line5 = folium.PolyLine(location5).add_to(m)
line6 = folium.PolyLine(location6).add_to(m)
line7 = folium.PolyLine(location7).add_to(m)

m

In [11]:
data_list = dataframe[['lat', 'lng', 'Data']].values.tolist()  # 在dataframe变量中提取出热力图所需信息
HeatMap(data_list, radius=15, gradient={0.3: 'blue', 0.6: 'lime', 0.8: 'yellow', 0.95: 'red'}).add_to(m)  # 将热力图加载到地图上
m

该热力图可对纵向光纤监测数据分布进行定性展示，例如红色的位置表示应力值较大，蓝色和绿色表示应力值较小。

图像显示效果不够好的原因可能有：监测数据不够长，使用的管道数据只有200m，而隧道总长为4-5km，这里没有拼接几次数据，仅用作初步尝试；这里忽略了跳线的长度以及光纤经过拐点的情况，之后会将其考虑进去，继续完善；隧道轮廓及两侧车站轮廓不够美观，轮廓线会盖在热力图上面，这一点目前还未找到较好的解决方式。

另外，若在PyQt5开发的界面中使用此功能，则无法实现双向的调用，即：只能在主要框架下调用地图，使地图承载并显示监测数据信息，而无法获取地图上的信息到主要框架中，例如点击地图上某点，无法获取其经纬度到自主编写的程序中。这一问题目前还未找到较好的解决方式。

请老师批评指正。